# Iris Flower - Batch Prediction


In this notebook we will, 

1. Load the batch inference data that arrived in the last 24 hours
2. Predict the first Iris Flower found in the batch
3. Write the ouput png of the Iris flower predicted, to be displayed in Github Pages.

In [2]:
import pandas as pd
import hopsworks
import joblib

project = hopsworks.login()
fs = project.get_feature_store()

2024-12-30 14:11:29,653 INFO: Initializing external client
2024-12-30 14:11:29,655 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-30 14:11:34,203 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207459


In [4]:
mr = project.get_model_registry()
model = mr.get_model("iris", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/iris_model.pkl")

We are downloading the 'raw' iris data. We explicitly do not want transformed data, reading for training. 

So, let's download the iris dataset, and preview some rows. 

Note, that it is 'tabular data'. There are 5 columns: 4 of them are "features", and the "variety" column is the **target** (what we are trying to predict using the 4 feature values in the target's row).

In [5]:
feature_view = fs.get_feature_view(name="iris", version=1)

Now we will do some **Batch Inference**. 

We will read all the input features that have arrived in the last 24 hours, and score them.

In [6]:
import datetime
from PIL import Image

#the batch data is the feature data, i.e without the labels
batch_data = feature_view.get_batch_data()

y_pred = model.predict(batch_data)

y_pred

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.90s) 


array(['Setosa', 'Versicolor', 'Setosa', 'Setosa', 'Setosa', 'Versicolor',
       'Virginica', 'Setosa', 'Virginica', 'Setosa', 'Setosa', 'Setosa',
       'Versicolor', 'Setosa', 'Versicolor', 'Virginica', 'Setosa',
       'Virginica', 'Virginica', 'Virginica', 'Versicolor', 'Virginica',
       'Setosa', 'Setosa', 'Setosa', 'Versicolor', 'Virginica',
       'Versicolor', 'Versicolor', 'Virginica', 'Virginica', 'Setosa',
       'Setosa', 'Setosa', 'Versicolor', 'Setosa', 'Versicolor',
       'Versicolor', 'Virginica', 'Versicolor', 'Versicolor',
       'Versicolor', 'Versicolor', 'Virginica', 'Versicolor', 'Setosa',
       'Virginica', 'Virginica', 'Setosa', 'Setosa', 'Versicolor',
       'Versicolor', 'Virginica', 'Setosa', 'Virginica', 'Setosa',
       'Versicolor', 'Versicolor', 'Virginica', 'Setosa', 'Virginica',
       'Setosa', 'Setosa', 'Virginica', 'Setosa', 'Versicolor', 'Setosa',
       'Virginica', 'Versicolor', 'Setosa', 'Virginica', 'Versicolor',
       'Virginica', 'Virgin

In [7]:
batch_data['sepal_length']

0      5.000000
1      6.600000
2      5.100000
3      5.100000
4      5.400000
         ...   
147    5.600000
148    6.800000
149    6.000000
150    7.768135
151    5.732203
Name: sepal_length, Length: 152, dtype: float64

Batch prediction output is the last entry in the batch - it is output as a file 'latest_iris.png'

In [6]:
#image of the predicted flower, the latest data added to the feature store
flower = y_pred[y_pred.size-1]
flower_img = "assets/" + flower + ".png"
img = Image.open(flower_img)            

img.save("../../assets/latest_iris.png")

In [8]:
iris_fg = fs.get_feature_group(name="iris", version=1)
df = iris_fg.read()
df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.98s) 


,sepal_length,sepal_width,petal_length,petal_width,variety,uuid
0,5.000000,3.300000,1.400000,0.200000,Setosa,48fe1330-1f4f-4526-8bb6-2f3592ab7b61
1,6.600000,3.000000,4.400000,1.400000,Versicolor,9b23506a-83b3-47fd-ac8f-8acfa716837d
2,5.100000,3.500000,1.400000,0.300000,Setosa,18383b75-a81b-4527-9cde-59b047a66bca
3,5.100000,3.700000,1.500000,0.400000,Setosa,0dd9d1fb-1d52-4df5-a6f2-b2c1a49c5db0
4,5.400000,3.400000,1.700000,0.200000,Setosa,eb0183b4-f3a2-4248-9b8b-a661f1bbcd54
...,...,...,...,...,...,...
147,5.600000,2.700000,4.200000,1.300000,Versicolor,61f98edb-730f-456a-89de-80e918cfa3b4
148,6.800000,2.800000,4.800000,1.400000,Versicolor,597a07b1-db33-4ce0-8ba7-606d244857b9
149,6.000000,2.900000,4.500000,1.500000,Versicolor,95452fe8-cfcf-483e-b678-0547fce114af
150,7.768135,3.645254,4.881264,1.835901,Virginica,87f8228b-9ebf-4783-97f1-c603b85ce431


In [9]:
label = df.iloc[-1]["variety"]
label

'Versicolor'

In [10]:
#image of the actual flower
label_flower = "assets/" + label + ".png"

img = Image.open(label_flower)            

img.save("../../assets/actual_iris.png")

In [10]:
import pandas as pd

monitor_fg = fs.get_or_create_feature_group(name="iris_predictions",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Iris flower Prediction/Outcome Monitoring"
                                 )

# Clear the contents of the feature group
# monitor_fg = fs.get_feature_group(name="iris_predictions", version=1)

# monitor_fg.delete()

# print("Feature group contents cleared successfully!")

In [11]:
from datetime import datetime
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    's_length': [float(df.iloc[-1]['sepal_length'])],  # Convert to scalar
    's_width': [float(df.iloc[-1]['sepal_width'])],
    'p_length': [float(df.iloc[-1]['petal_length'])],
    'p_width': [float(df.iloc[-1]['petal_width'])],
    'prediction': [flower],  # Ensure 'flower' is a scalar value
    'label': [label],        # Ensure 'label' is a scalar value
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: iris_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207459/jobs/named/iris_predictions_1_offline_fg_materialization/executions


(Job('iris_predictions_1_offline_fg_materialization', 'SPARK'), None)

In [12]:
# history_df = monitor_fg.read()
# history_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.02s) 


,s_length,s_width,p_length,p_width,prediction,label,datetime
0,5.700000,3.800000,1.700000,0.300000,Virginica,Virginica,"12/28/2024, 15:40:30"
1,5.700000,3.800000,1.700000,0.300000,Virginica,Virginica,"12/28/2024, 15:43:10"
2,6.402626,2.825648,6.617902,1.661238,Virginica,Virginica,"12/28/2024, 15:43:33"
3,6.402626,2.825648,6.617902,1.661238,Virginica,Virginica,"12/28/2024, 15:47:56"


In [13]:
# import dataframe_image as dfi

# df_recent = history_df.tail(5)
 
# # If you exclude this image, you may have the same iris_latest.png and iris_actual.png files
# # If no files have changed, the GH-action 'git commit/push' stage fails, failing your GH action (last step)
# # This image, however, is always new, ensuring git commit/push will succeed.
# dfi.export(df_recent, '../../assets/df_recent.png', table_conversion = 'matplotlib')

In [19]:
# from sklearn.metrics import confusion_matrix

# predictions = history_df[['prediction']]
# labels = history_df[['label']]

# results = confusion_matrix(labels, predictions)
# print(results)

[[0 1]
 [0 0]]


In [20]:
from matplotlib import pyplot
import seaborn as sns

# Only create the confusion matrix when our iris_predictions feature group has examples of all 3 iris flowers
if results.shape == (3,3):

    df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                         ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

    cm = sns.heatmap(df_cm, annot=True)

    fig = cm.get_figure()
    fig.savefig("../../assets/confusion_matrix.png") 
    df_cm
else:
    print("Run the batch inference pipeline more times until you get 3 different iris flowers")    

Run the batch inference pipeline more times until you get 3 different iris flowers
